In [11]:
import cv2
import os
from PIL import Image
import numpy as np
import pandas as pd

## Exportación frames

In [5]:
vidcap = cv2.VideoCapture('../data/ruido_videos/merge.mp4')


In [6]:
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("../data/ruido_fotos/pic"+str(count)+".jpg", image)
    return hasFrames

sec = 0
frameRate = 1/30
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

## Dataset a partir de imagen

In [15]:

from resizeimage import resizeimage

for i in range(1,6242):
    with open('../data/ruido_fotos/pic{}.jpg'.format(i), 'r+b') as f:
        with Image.open(f) as image:
            smaller = resizeimage.resize_cover(image, [28, 28])
            cover.save('../data/resized_fotos/pic{}.jpeg'.format(i), image.format)


In [16]:
for i in range(1,6242):
    with open('../data/resized_fotos/pic{}.jpeg'.format(i), 'r+b') as f:
        with Image.open(f) as image:
            img = image.convert('L')
            img.save('../data/resized_fotos/pic{}.jpeg'.format(i))


### Imágenes (28,28) a array

In [21]:
noise = pd.DataFrame()
for i in range(1,6242):
    with open('../data/resized_fotos/pic{}.jpeg'.format(i), 'r+b') as f:
        with Image.open(f) as image:
            arr = np.array(image)
            noise = noise.append(pd.Series(arr.flatten()), ignore_index=True)


In [22]:
noise["label"] = "other"

In [23]:
noise

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,59.0,133.0,25.0,71.0,61.0,122.0,150.0,143.0,123.0,94.0,...,189.0,204.0,212.0,211.0,201.0,219.0,208.0,187.0,121.0,other
1,108.0,43.0,57.0,57.0,82.0,138.0,173.0,128.0,88.0,114.0,...,200.0,190.0,186.0,208.0,247.0,214.0,221.0,184.0,116.0,other
2,18.0,51.0,69.0,74.0,123.0,172.0,145.0,101.0,90.0,138.0,...,210.0,190.0,206.0,223.0,197.0,205.0,179.0,179.0,96.0,other
3,38.0,63.0,72.0,147.0,180.0,147.0,100.0,104.0,129.0,57.0,...,195.0,204.0,213.0,222.0,209.0,183.0,167.0,144.0,82.0,other
4,38.0,63.0,72.0,147.0,180.0,147.0,100.0,104.0,129.0,57.0,...,195.0,204.0,213.0,222.0,209.0,183.0,167.0,144.0,82.0,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6236,69.0,120.0,116.0,81.0,110.0,118.0,143.0,144.0,138.0,124.0,...,70.0,68.0,68.0,69.0,71.0,80.0,81.0,85.0,43.0,other
6237,63.0,115.0,116.0,85.0,112.0,115.0,141.0,146.0,143.0,127.0,...,70.0,68.0,68.0,71.0,73.0,78.0,83.0,86.0,42.0,other
6238,64.0,115.0,115.0,83.0,110.0,113.0,140.0,145.0,143.0,116.0,...,71.0,68.0,68.0,70.0,72.0,76.0,83.0,88.0,44.0,other
6239,63.0,111.0,117.0,87.0,100.0,125.0,130.0,152.0,142.0,126.0,...,71.0,68.0,68.0,70.0,71.0,76.0,82.0,88.0,43.0,other


In [24]:
noise.to_csv("../data/processed/background.csv", index=False)

## Mejoro dataset con otros gestos como ruido

In [66]:
train = pd.read_csv("../data/sign_mnist/sign_mnist_train.csv")

In [67]:
newLabels = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G",
            7: "H", 8: "I", 10: "K", 11: "L", 12: "M", 13: "N",
            14: "O", 15: "P", 16: "Q", 17: "R", 18:"S",
            19: "T", 20:"U", 21:"V", 22:"W", 23:"X",
            24:"Y"}

train["label"] = train["label"].map(newLabels)

In [95]:
signs = ["A", "B", "F", "T", "V", "Y"]

morenoise = train[(train.label != "A") & (train.label != "B") & (train.label != "F") & (train.label != "T")
              & (train.label != "V") & (train.label != "Y")]


In [98]:
morenoise.label = "other"

In [99]:
morenoise

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,other,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,other,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,other,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,other,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,other,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,other,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,other,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,other,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,other,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [100]:
morenoise.to_csv("../data/processed/morenoise.csv", index=False)

In [115]:
data = pd.read_csv("../data/processed/data.csv")

In [101]:
noisyData = pd.concat([morenoise, data])

In [104]:
noisyData.label.value_counts()

other    26238
A         1457
F         1451
Y         1450
B         1442
T         1434
V         1428
Name: label, dtype: int64

In [105]:
noisyData.to_csv("../data/processed/noisyData.csv", index=False)

### Demasiado ruido - dataset más balanceado

In [121]:
morenoise.sample(n=2700, random_state=1)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
7720,other,208,211,211,212,212,213,213,213,214,...,192,198,201,176,129,132,151,154,158,162
242,other,177,178,177,177,178,178,178,178,178,...,191,202,203,167,117,101,91,86,80,75
25098,other,184,183,184,185,184,184,185,184,185,...,207,205,204,205,205,203,202,200,199,199
14865,other,180,182,184,187,189,192,193,194,194,...,123,106,113,218,230,229,230,229,226,225
281,other,162,168,174,178,181,184,186,188,190,...,15,48,42,19,29,35,25,24,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,other,205,205,206,207,208,209,209,209,210,...,228,227,226,226,223,222,222,222,220,220
15326,other,136,138,139,139,139,139,140,139,139,...,190,186,165,122,64,28,19,22,26,104
24650,other,135,138,141,145,148,151,154,156,158,...,90,69,160,219,208,210,211,208,178,200
7574,other,185,185,186,184,184,185,184,183,182,...,55,35,47,53,33,43,35,37,59,147


In [122]:
data[data.label == "other"].sample(n=5500, random_state=1)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,label
7865,106.0,167.0,126.0,102.0,132.0,86.0,41.0,68.0,64.0,101.0,...,75.0,87.0,91.0,84.0,73.0,100.0,114.0,109.0,47.0,other
9992,130.0,185.0,178.0,175.0,169.0,182.0,179.0,177.0,179.0,179.0,...,85.0,80.0,88.0,82.0,81.0,79.0,75.0,84.0,46.0,other
9411,76.0,92.0,92.0,86.0,97.0,115.0,132.0,148.0,139.0,154.0,...,94.0,92.0,89.0,84.0,80.0,79.0,70.0,75.0,39.0,other
8645,72.0,118.0,182.0,171.0,240.0,154.0,10.0,84.0,55.0,112.0,...,84.0,70.0,53.0,119.0,166.0,144.0,113.0,141.0,75.0,other
9140,130.0,184.0,198.0,184.0,192.0,141.0,47.0,139.0,181.0,188.0,...,82.0,82.0,81.0,80.0,78.0,67.0,65.0,66.0,26.0,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12122,47.0,145.0,83.0,25.0,32.0,83.0,79.0,149.0,250.0,250.0,...,39.0,53.0,68.0,57.0,100.0,141.0,136.0,146.0,68.0,other
7928,40.0,73.0,82.0,74.0,72.0,81.0,44.0,71.0,166.0,224.0,...,139.0,137.0,146.0,143.0,150.0,158.0,160.0,172.0,89.0,other
10188,98.0,145.0,136.0,140.0,134.0,140.0,134.0,136.0,135.0,135.0,...,141.0,142.0,143.0,143.0,143.0,146.0,144.0,157.0,79.0,other
9523,168.0,248.0,253.0,241.0,242.0,255.0,242.0,255.0,220.0,64.0,...,61.0,72.0,82.0,83.0,81.0,68.0,87.0,110.0,36.0,other


In [123]:
noise = pd.concat([morenoise.sample(n=2700, random_state=1), data[data.label == "other"].sample(n=5500, random_state=1)])

In [124]:
noise.to_csv("../data/processed/betterNoise.csv", index=False)

In [125]:
data[data.label != "other"]

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,label
0,197.0,195.0,196.0,195.0,197.0,196.0,195.0,196.0,196.0,196.0,...,65.0,182.0,213.0,211.0,212.0,212.0,213.0,213.0,213.0,A
1,142.0,144.0,144.0,146.0,147.0,149.0,150.0,151.0,153.0,154.0,...,179.0,179.0,180.0,181.0,182.0,182.0,182.0,183.0,183.0,A
2,198.0,200.0,201.0,200.0,199.0,198.0,198.0,197.0,198.0,199.0,...,99.0,99.0,98.0,99.0,98.0,100.0,100.0,101.0,100.0,A
3,231.0,232.0,234.0,237.0,238.0,241.0,243.0,244.0,248.0,249.0,...,66.0,199.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,A
4,147.0,149.0,150.0,152.0,153.0,153.0,152.0,153.0,154.0,154.0,...,165.0,166.0,165.0,166.0,169.0,167.0,133.0,135.0,140.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14166,63.0,74.0,85.0,94.0,101.0,105.0,105.0,111.0,117.0,120.0,...,176.0,175.0,175.0,175.0,176.0,176.0,176.0,176.0,175.0,Y
14167,157.0,158.0,160.0,162.0,164.0,166.0,167.0,168.0,168.0,169.0,...,190.0,188.0,188.0,187.0,187.0,186.0,185.0,184.0,182.0,Y
14168,171.0,171.0,171.0,171.0,171.0,172.0,172.0,172.0,172.0,171.0,...,119.0,116.0,107.0,87.0,63.0,62.0,66.0,60.0,59.0,Y
14169,178.0,178.0,178.0,179.0,178.0,179.0,179.0,179.0,179.0,178.0,...,115.0,112.0,100.0,77.0,51.0,46.0,48.0,41.0,39.0,Y


In [126]:
data_2 = pd.concat([data[data.label != "other"], noise])

In [128]:
data_2.label.value_counts()

other    8200
A        1457
F        1451
Y        1450
B        1442
T        1434
V        1428
Name: label, dtype: int64

In [130]:
data_2.to_csv("../data/processed/data_2.csv", index=False)